1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM 질문과 같이 전달

1. 패키지 설치

In [ ]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-chroma

2. Knowledge Base 구성을 위한 데이터 생성

- RecursiveCharacterTextSplitter를 활용한 데이터 chunking
  - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
  - 비용 감소와 답변 생성시간 감소의 효과
  - LangChain에서 다양한 TextSplitter들을 제공
- chunk_size 는 split 된 chunk의 최대 크기
- chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(
    model="embedding-query"
)

In [3]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때
# database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

3. 답변 생성을 위한 Retrieval

Chroma에 저장한 데이터를 유사도 검색(similarity_search())를 활용해서 가져옴

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)
# retrieved_docs

4. Augmentation을 위한 Prompt 활용

Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [5]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model='solar-pro')

In [6]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/crazyatom/.pyenv/versions/3.11.11/envs/llm-application/lib/python3.11/site-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
  - RetrievalQA는 create_retrieval_chain으로 대체됨
  - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음

In [7]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [8]:
# ai_message = qa_chain({"query": query})

# 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [9]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'result': '정확한 금액을 알 수 없습니다.'}